In [1]:
import numpy as np
from numpy.random import default_rng
from scipy.integrate import odeint
from scipy.stats import rv_continuous
import matplotlib.pyplot as plt

In [8]:
# Define classes
class Agent:
    def __init__(self, opinion, activity):
        self.x=opinion
        self.a=activity
        #self.active=False
        
    def setOpinion(opinion):
        self.x=opinion   
        
    def getOpinion(self):
        return self.x
    
    def setActivity(activity):
        self.a=activity
        
class Model:
    def __init__(self, N):
        
        x = np.linspace(-1,1,N)
        PowerDistribution = PowerLaw(a=epsilon)
        a = PowerDistribution.rvs(size=N)
        self.network = []
        for i in range(N):
            self.network.append(Agent(x[i], a[i]))
            
    def getNetwork(self):
        return self.network
    
    def getAllOpinions(self):
        allOpinions = np.zeros(len(self.network))
        for i in range(len(self.network)):
            allOpinions[i] = self.network[i].getOpinion()
        return allOpinions

class PowerLaw(rv_continuous):
    """Activitiy sampling distribution identical to Baumann et al.
    It models the intuitive assumption, that by far most people
    are hardly posting on social media, and even less people being
    very active. 
    

    For Reference, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html#scipy.stats.rv_continuous"""
    # define probability distribution as the paper's "F(a)"
    def _pdf(self, x):
        return ((1-gamma)/(1-epsilon**(1-gamma)))*x**(-1*gamma)


In [ ]:
# Define methods

# Implement differential eq. 'dx/dt = [...]' as 'f(t) = [...]' 
def diffEq(x, t, alpha, K):
    sol = np.zeros(N)
    for i in range(N):
        sum_tmp = 0
        for j in range(N):
            sum_tmp = sum_tmp+A[i][j]*np.tanh(alpha*x[j])
        sol[i] = -x[i]+K*sum_tmp
    return sol

def p(i, j, x, beta):
    sum_tmp = 0
    for j_tmp in range(len(x)):
        if x[i] != x[j_tmp]:
            sum_tmp = sum_tmp+abs(x[i]-x[j_tmp])**(-beta)
    return (abs(x[i]-x[j])**(-beta))/sum_tmp

def A_gen(x, beta):
    rng = default_rng()
    A = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i != j:
                A[i][j] = p(i,j, x, beta)

    for i in range(N):
        m = 0
        while m < 10:
            for j in range(N):
                if A[i][j] > rng.random() and m<10:
                    A[i][j] = 1.
                    m = m+1
                    print("    m=", m)
                else:
                    A[i][j] = 0
        return A

In [ ]:
# Runge-Kutta 4 Implementation
# Define fixed parameters
t_current = 0
tMax = 0.1
dt = 0.01
N = 100

K=3
# Define different alpha, beta values as arrays from which to pick for each graph
alphas = np.array([.05, 3, 3])
betas = np.array([2, 0, 3])
alpha_current = alphas[0]
beta_current = betas[0]

# Fixed activity-driving (AD) parameters
#m = 10
epsilon = 0.01
gamma = 2.1
#r = 0.5


model = Model(N)
opinionStorage = model.getAllOpinions()

while t_current < tMax:
    currentOpinions = model.getAllOpinions()
    t_RK4 = np.linspace(t_current, t_current+dt, 4)
    A = A_gen(currentOpinions, beta_current)
    solODE = odeint(diffEq, currentOpinions, t_RK4, args=(alpha_current, K))
    # Append second column of solODE, as first one is filled with starting values
    opinionStorage = np.vstack([opinionStorage, solODE[3]])
    print(t_current)
    t_current = t_current + dt

1
2
3
4
5
6
7
8
9
10
0


In [ ]:
file = open('example.txt', 'w')
np.savetxt('example.txt', opinionStorage)
file.close()

In [ ]:
opinionStorage.T

In [ ]:
len(opinionStorage)

In [ ]:
color = 'b'
#iterate through network and draw lines


for i in opinionStorage.T:
    if sum(i) < 0:
        color = 'r'
    else:
        color ='b'
    print(i)   
    plt.plot(i, color)

    
plt.xlabel('Time')
plt.ylabel('Opinion')
plt.title('Figure 1')

plt.ylim([-1, 1])
plt.xticks([0, 5, 10])
plt.yticks([-5, 0, 5])
plt.show()

In [ ]:
len(xneu)